# ENUM_BANK의 주소를 transferwise 에서 가져오는 Crawler
## 수정해야 하는 곳
### 3번셀에서 csv파일명 수정
### 5번셀에서 크롬드라이버의 경로

## 필요한 라이브러리들 인젝션

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import time

## 셀레늄에 헤더정보 설정 및 헤더리스 설정

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument("disable-gpu")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36")
chrome_options.add_argument("lang=ko_KR") # 한국어!

## GIVEN
### transferwise 에서 csv에 있는 은행명으로 검색할 수 있도록 준비

In [3]:
# 크롤링할 csv 파일 만들어서 이곳에서 교체
target_country = 'Bangladesh'
df_bank = pd.read_csv(target_country + '-Bank.csv')
target_list = df_bank.descr

## 사전 테스트

In [4]:
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
driver.get('https://transferwise.com/gb/swift-codes/banks?country=0&city=0&name=THE+TOHOKU+BANK')
soup = BeautifulSoup(driver.page_source,'html.parser')

In [5]:
tmp_bank = soup.select('div div div div div div ul li div a')
tmp_addr = soup.select('div div div div div div ul li small span')

print(tmp_bank[0].text, ' : ', tmp_addr[1].text)

The Tohoku Bank, Ltd  :  3-1 Uchimaru Morioka 020 0023, 


In [6]:
soup.find(id='country-selector')

<select class="form-control selectpicker" data-live-search="true" id="country-selector" name="country" tabindex="-98">
<option value="0">All countries</option>
<option value="135">Australia</option>
<option value="136">Austria</option>
<option value="142">Belgium</option>
<option value="148">Brazil</option>
<option value="149">Bulgaria</option>
<option value="154">Canada</option>
<option value="158">Chile</option>
<option value="159">China</option>
<option value="160">Colombia</option>
<option value="163">Croatia</option>
<option value="165">Cyprus</option>
<option value="166">Czech Republic</option>
<option value="167">Denmark</option>
<option value="174">Estonia</option>
<option value="177">Finland</option>
<option value="178">France</option>
<option value="181">Georgia</option>
<option value="182">Germany</option>
<option value="184">Gibraltar</option>
<option value="185">Greece</option>
<option value="195">Hong Kong</option>
<option value="196">Hungary</option>
<option value="197">

## WHEN

In [7]:
addr = []
bank = []
url = 'https://transferwise.com/gb/swift-codes/banks?country=313&city=0&name='
for idx, bank_name in enumerate(target_list) :
    print(idx)
    tmp_str = url + bank_name
    driver.get(tmp_str)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    bank.append(soup.select('div div div div div div ul li div a')[0].text)
    addr.append(soup.select('div div div div div div ul li small span')[1].text)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


In [9]:
result = {'bank':bank, 'descr':target_list, 'addr':addr}
df_result = pd.DataFrame(result)
df_result.head()

,bank,descr,addr
0,Mutual Trust Bank Limited,Mutual Trust Bank,"37 Agrabad C/a Chittagong,"
1,Agrani Bank Limited,Agrani Bank,"Agrani Bank Bhavan 1000 Dhaka,"
2,Al-arafah Islami Bank Ltd.,Al-Arafah Islami Bank,"Rahman Mension 1000 Dhaka,"
3,Agrani Bank Limited,AB Bank,"Agrani Bank Bhavan 1000 Dhaka,"
4,Bangladesh Krishi Bank,Bangladesh Bank,"Krishi Bank Bhaban 1000 Dhaka,"


In [10]:
df_result.to_csv(target_country + '.csv', header=False, index=False)